In [3]:
#do ping manually, from h1 to h4, without running mininet from start.
#ps aux | grep mininet:h1 | awk ' NR==1 { print $2 }'
#echo test@irciss008 | sudo -S mnexec -a 1962591 ping -c 2 10.0.0.4

In [ ]:
import subprocess
import time
import threading
import import_ipynb
import re
import ONOS_End_to_End_IBN_Functions as onos_ibn
sudo_password = "your_password"

In [11]:
def extract_host_and_ip (flow_data):

    # Extract first flow rule
    flow = flow_data["flows"][0]

    # Extract source and destination IPs
    src_ip = None
    dst_ip = None

    for criterion in flow["selector"]["criteria"]:
        if criterion["type"] == "IPV4_SRC":
            src_ip = criterion["ip"].split("/")[0]  # Remove /32 subnet
        if criterion["type"] == "IPV4_DST":
            dst_ip = criterion["ip"].split("/")[0]

    #print(f"Extracted Source IP: {src_ip}")  # Expected Output: 10.0.0.1 (h1)
    #print(f"Extracted Destination IP: {dst_ip}")  # Expected Output: 10.0.0.4 (h4)

    # Convert IPs to Host Names (Assuming Static Mapping)
    ip_to_host = {
        "10.0.0.1": "h1",
        "10.0.0.2": "h2",
        "10.0.0.3": "h3",
        "10.0.0.4": "h4"
    }

    src_host = ip_to_host.get(src_ip, "Unknown")
    dst_host = ip_to_host.get(dst_ip, "Unknown")

    #print(f"Source Host: {src_host}")  # Expected Output: h1
    #print(f"Destination Host: {dst_host}")  # Expected Output: h4
    return src_host, dst_host, src_ip, dst_ip

In [12]:
def execute_command(command):
    """
    Runs a command with sudo password automation.
    """
    full_command = f"echo {sudo_password} | sudo -S {command}"
    try:
        result = subprocess.run(full_command, shell=True, capture_output=True, text=True)
        if result.returncode == 0:
            return result.stdout.strip()
        else:
            raise Exception(f"Error executing command: {result.stderr.strip()}")
    except Exception as e:
        return str(e)

In [13]:
def execute_command_full(command, timeout):
    """Execute a command and capture full output, waiting for all responses."""
    try:
        process = subprocess.Popen(
            command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )

        output_lines = []
        start_time = time.time()

        while True:
            line = process.stdout.readline()
            if line:
                output_lines.append(line.strip())  # Store the output line
                #print(line.strip())  # Print live output (optional)

            # Check if process has completed
            if process.poll() is not None:
                break

            # Timeout check
            if time.time() - start_time > timeout:
                process.terminate()  # Stop process if timeout occurs
                raise TimeoutError(f"Command timed out after {timeout} seconds")

        # Capture remaining output
        remaining_output, _ = process.communicate()
        if remaining_output:
            output_lines.append(remaining_output.strip())

        return "\n".join(output_lines)

    except Exception as e:
        return str(e)

In [14]:
def onos_assurance (src_host, dst_host, src_ip, dst_ip, ping_count=2):
    
    #host_count_cmd = "ps aux | grep mininet:h | grep root | wc -l"
    #host_count = int(subprocess.check_output(host_count_cmd, shell=True, text=True))-1 #if you want to dynamically find host count
    #print("Total Number of hosts: ",host_count)

    host_pid_cmd = "ps aux | grep mininet:" + src_host + " | awk ' NR==1 { print $2 }'"
    host_pid = int(subprocess.check_output(host_pid_cmd, shell=True, text=True))
    print("\nProcess ID of source host ", src_host,": ", host_pid)

    #dst_host_pid_cmd = "ps aux | grep mininet:" + dst_host  + " | awk ' NR==1 { print $2 }'"
    #dst_host_ip_cmd = "echo " + sudo_password + " | sudo -S mnexec -a $(" + dst_host_pid_cmd + ") ifconfig | grep inet | awk ' NR==1 { print $2 }'"
    #dst_host_ip = execute_command(dst_host_ip_cmd) #If you want to dynamically find a host's IP
    
    dst_host_ip = dst_ip #if you want to use ONOS flow destination IP 
    print("IP of destination host ",dst_host,": ", dst_host_ip)

    timeout=15

    ping_command = f"echo " + sudo_password + f" | sudo -S mnexec -a {host_pid} ping -c {ping_count} {dst_host_ip}"
    #print(execute_command(ping_command))
    output = execute_command_full(ping_command, timeout)
    if "100% packet loss" in output:
        print(f"\nIntent Effective. Traffic from {src_host} to {dst_host} is Blocked.")

    elif "0% packet loss" in output:
        print(f"\nIntent Not Effective. Traffic from {src_host} to {dst_host} is Not BLOCKED.")
    else:
        print("\nIntent Not Effective. Invalid Assurance Report Received. Check command for ping execution.\n")
        print(output)

In [ ]:
def extract_switch_id(intent: str):
    """
    Extract the switch ID from a natural language intent.
    
    Parameters:
        intent (str): The natural language intent.
    
    Returns:
        str: Extracted switch ID (e.g., 'openflow:1') or None if not found.
    """
    # Mapping of ordinal words to numeric values
    ordinals = {
        "first": 1,
        "second": 2,
        "third": 3,
        "fourth": 4,
        "fifth": 5,
        "sixth": 6,
        "seventh": 7,
        "eighth": 8,
        "ninth": 9,
        "tenth": 10
    }

    # Match patterns like 'openflow:1'
    match = re.search(r'openflow[:\s](\d+)', intent, re.IGNORECASE)
    if match:
        return f"openflow:{match.group(1)}"

    # Match patterns like 'switch 1', 'router 2', 'node 3'
    match = re.search(r'\b(?:switch|router|node|device)(?:\s*number)?\s*(\d+)', intent, re.IGNORECASE)
    if match:
        return f"openflow:{match.group(1)}"

    # Match ordinal words (e.g., 'fourth switch', 'second router')
    match = re.search(r'\b(?:switch|router|node|device)\s*(\w+)', intent, re.IGNORECASE)
    if match:
        ordinal_word = match.group(1).lower()
        if ordinal_word in ordinals:
            return f"openflow:{ordinals[ordinal_word]}"

    # Match standalone ordinal words (e.g., 'fourth' without 'switch')
    for word, number in ordinals.items():
        if word in intent.lower():
            return f"openflow:{number}"

    return None


def get_switch_port_mapping():
    try:
        # Commands to list port and QoS configurations
        list_ports_command = "sudo -S ovs-vsctl list port"
        list_qos_command = "sudo -S ovs-vsctl list qos"
        # Fetch port and QoS data
        ports_output = execute_command(list_ports_command)
        qos_output = execute_command(list_qos_command)

        # Parse QoS data into a dictionary
        qos_mapping = {}
        current_qos = None
        for line in qos_output.splitlines():
            if line.startswith("_uuid"):
                current_qos = line.split(":")[1].strip()
            elif line.startswith("queues") and current_qos:
                qos_mapping[current_qos] = line.split(":")[1].strip()

        # Create a dictionary to store switch-to-port mapping
        switch_port_dict = {}

        # Parse ports data and check for QoS
        current_port = None
        for line in ports_output.splitlines():
            if line.startswith("name"):
                current_port = line.split(":")[1].strip()
            elif line.startswith("qos") and "[]" not in line and current_port:
                qos_uuid = line.split(":")[1].strip()

                # Extract the OpenFlow switch ID and port number
                if "-" in current_port:
                    switch, port = current_port.split("-")
                    switch_id = f"openflow:{switch[1:]}"  # e.g., "s1" -> "openflow:1"
                    port_number = port[3:]  # e.g., "eth2" -> "2"

                    # Add to dictionary
                    if switch_id not in switch_port_dict:
                        switch_port_dict[switch_id] = []
                    switch_port_dict[switch_id].append(port_number)

                current_port = None

        return switch_port_dict

    except Exception as e:
        print(f"Error: {e}")
        return {}
    
def extract_port_number(text: str):
    """
    Extract the Ethernet port number from a natural language text.
    
    Parameters:
        text (str): The input text containing the port reference.
    
    Returns:
        int: Extracted port number or None if not found.
    """
    # Mapping of ordinal words to numeric values
    ordinals = {
        "first": 1,
        "second": 2,
        "third": 3,
        "fourth": 4,
        "fifth": 5,
        "sixth": 6,
        "seventh": 7,
        "eighth": 8,
        "ninth": 9,
        "tenth": 10
    }

    # Match explicit numbers after keywords
    match = re.search(r'\b(?:port|interface|output\s+node\s+connector|ethernet)\s*(\d+)', text, re.IGNORECASE)
    if match:
        return int(match.group(1))

    # Match ordinal words (e.g., 'second port', 'third interface')
    match = re.search(r'\b(?:port|interface|output\s+node\s+connector|ethernet)\s*(\w+)', text, re.IGNORECASE)
    if match:
        ordinal_word = match.group(1).lower()
        if ordinal_word in ordinals:
            return ordinals[ordinal_word]

    # Match standalone ordinal words (e.g., 'second')
    for word, number in ordinals.items():
        if word in text.lower():
            return number

    return None

def ovs_port_exists(switch_name, port_name, sudo_password=sudo_password):
    import subprocess
    cmd = f"sudo -S ovs-vsctl list-ports {switch_name}"
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True, input=sudo_password + "\n")
    ports = result.stdout.split()
    return port_name in ports


def get_switch_name_from_device_id(device_id):
    # device_id is like 'openflow:2'
    idx = int(device_id.split(':')[1])
    return f"s{idx}onos"


def create_two_queue_for_switch(device_id, port, max_rate=10000000, queue_configs=None):
    """
    Creates queues dynamically for a specific switch and port.
    
    Parameters:
        switch (str): The name of the switch in 'openflow:X' format (e.g., 'openflow:4').
        port (int): The port number on the switch (e.g., 2).
        max_rate (int): Maximum rate for the QoS (default is 10000000).
        queue_configs (list): List of tuples specifying min-rate and max-rate for each queue (default is 2 queues).
    """
    if queue_configs is None:
        # Default to 2 queues with these configurations
        queue_configs = [
            (6000000, 6000000),  # Queue 0: min-rate and max-rate
            (4000000, 4000000)   # Queue 1: min-rate and max-rate
        ]

    # Construct the port name from the input
    #port_name = f"{switch.replace('openflow:', 's')}-eth{port}"
    switch_name = get_switch_name_from_device_id(device_id)
    port_name = f"{switch_name}-eth{port}"

    if not ovs_port_exists(switch_name, port_name):
        print(f"Port {port_name} does not exist on bridge {switch_name}!")
        return


    # Construct the QoS command for the specific switch and port
    qos_command = f"sudo -S ovs-vsctl -- set port {port_name} qos=@newqos -- --id=@newqos create qos type=linux-htb other-config:max-rate={max_rate}"
    for i, (min_rate, max_rate) in enumerate(queue_configs):
        qos_command += f" queues:{i}=@q{i}"
    for i, (min_rate, max_rate) in enumerate(queue_configs):
        qos_command += f" -- --id=@q{i} create queue other-config:min-rate={min_rate} other-config:max-rate={max_rate}"

    # Execute the command
    print(f"Running: {qos_command}")
    process = subprocess.Popen(qos_command, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stdout, stderr = process.communicate(input=f"{sudo_password}\n")
    if process.returncode == 0:
        print(f"Success:\n{stdout}")
    else:
        print(f"Error:\n{stderr}")


def create_two_queue_for_switch_handler(slicing_info):

    if 'use_queue' in slicing_info:         
            slicing_status = slicing_info['use_queue']
            slicing_switch_id = slicing_info['switch_id']
            slicing_queue_id = slicing_info['queue_id']
            slicing_port_id = slicing_info['port_id']

            if(slicing_status == 1):
                openflow_id = extract_switch_id(slicing_switch_id)
                switch_port_mapping = get_switch_port_mapping()
                print("\nCheckpoint*******Entering Slice/Queue Management\n\n******")

                port_number = extract_port_number(slicing_port_id)
                
                if openflow_id not in switch_port_mapping :
                    
                        print("\n\nQueue was not installed in ",openflow_id, "\nInstalling now on interface: ", port_number,"\n")
                        print("\nCheckpoint*******Entering queue creation\n\n******")

                        create_two_queue_for_switch(
                            device_id=openflow_id,  port=port_number,
                            queue_configs=[
                                (6000000, 6000000),  # Queue 0
                                (4000000, 4000000)   # Queue 1
                            ]
                            )   
                else:
                    if str(port_number) not in switch_port_mapping[openflow_id]:

                        print("\n\nQueue was not installed in ",openflow_id, " interface: ", port_number, "\nInstalling now...\n")
                        print("\nCheckpoint*******Entering queue creation\n\n******")

                        create_two_queue_for_switch(
                            device_id=openflow_id,  port=port_number,
                            queue_configs=[
                                (6000000, 6000000),  # Queue 0
                                (4000000, 4000000)   # Queue 1
                            ]
                            )
    

In [ ]:
intent = input().strip()
#intent = "in switch 4, install a firewall to block traffic from 10.0.0.2 to 10.0.0.4"
#intent = "in switch 4, traffic destined for 10.0.0.4 should pass through port 3"
#intent = "Using openflow switch number 1, forward UDP traffic on port 80 destined for 10.0.0.3 via interface 2, assigning it to queue 1 for prioritized handling."
#intent = "In switch 3, if the incoming traffic in port 1 is TCP traffic destined for port 80, then pass it via interface 2, assigning it to queue 0 for prioritized handling."
#intent = "In switch 3, if the incoming traffic in interface 1 is UDP traffic destined for port 80, then pass it via port 2, assigning it to queue 1 for prioritized handling."
#intent = "In switch 3, if the incoming traffic in port 2 is TCP traffic destined for port 80, then pass it via interface 1, assigning it to queue 0 for prioritized handling."
#intent = "In switch 3, if the incoming traffic in interface 2 is UDP traffic destined for port 80, then pass it via port 1, assigning it to queue 1 for prioritized handling."

slicing_info = onos_ibn.run_LLM_Slice(intent)

create_two_queue_for_switch_handler(slicing_info) #this creates queue in switch if needed

conflict_status, new_onos_flow_rule = onos_ibn.end_to_end_IBN(intent)

if (conflict_status == 0):
    start_time = time.time()
    src_host, dst_host, src_ip, dst_ip = extract_host_and_ip(new_onos_flow_rule)
    onos_assurance(src_host, dst_host, src_ip, dst_ip)
    elapsed_time = (time.time() - start_time)
    print("Time taken to verify intent: ",  round(elapsed_time,2))